In [1]:
import time
startTime = time.time()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pixstem.api as ps
import multiprocessing
import hyperspy.api as hs
%matplotlib qt

file = hs.load("input.blo")

In [2]:
def distance(x1, y1, x2, y2):
    return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))

def pos_dist(x1, y1, x2, y2):
    if (x2 > x1 and y2 > y1):
        return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
    else:
        return -1.0
def findCenter(im, peak):
    center = (0,0)
    maximum = 0
    for (x,y) in np.ndenumerate(peak):
        for (a, b) in y:
            if (int(a) < len(im) and int(b) < len(im) and im[int(a)][int(b)] > maximum):
                maximum = im[int(a)][int(b)]
                center = (b, a)
    return center
def multiprocessing_func(i, j):
    s = ps.PixelatedSTEM(hs.signals.Signal2D(file.inav[i, j]))
    imarray = np.array(s)
    s = s.rotate_diffraction(0,show_progressbar=False)
    ############################################################################################################################
    st = s.template_match_disk(disk_r=5, lazy_result=False, show_progressbar=False)
    peak_array = st.find_peaks(lazy_result=False, show_progressbar=False)
    peak_array_com = s.peak_position_refinement_com(peak_array, lazy_result=False, show_progressbar=False)
    s_rem = s.subtract_diffraction_background(lazy_result=False, show_progressbar=False)
    peak_array_rem_com = s_rem.peak_position_refinement_com(peak_array, lazy_result=False, show_progressbar=False)
    ############################################################################################################################
    center = findCenter(imarray, peak_array_rem_com)

    # finds the specific spot and adding that distance to the array
    posDistance = 0
        
    for (x,y) in np.ndenumerate(peak_array_rem_com):
        for (a, b) in y:
            distances.append(round(distance(center[0], center[1], b, a), 2))

In [3]:
ROW = 10
COL = 10

manager = multiprocessing.Manager()
distances = manager.list()

for i in range(ROW):
    print(i)
    processes = []
    for j in range(COL):
        p = multiprocessing.Process(target=multiprocessing_func, args=(i, j,))
        processes.append(p)
        p.start()

    for process in processes:
        process.join()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [4]:
def count(x, i):
    ctr = 0
    for x in distances:
        if i <= x < i + 0.01: # change the 0.01 to whatever number you use above in the first comment
            ctr += 1
    return ctr

In [6]:
INTERVAL = 0.01

distances.sort()
x_pos = np.arange(np.min(distances), np.max(distances), INTERVAL) # this 0.01 is the distance between each x-axis label. So for example it goes 1.0, 1.01, 1.02 1.03...
x_pos = [round(num, 2) for num in x_pos]
y_pos = np.arange(len(x_pos))

pool = multiprocessing.Pool()
counts = [pool.apply(count, args=(x, INTERVAL,)) for x in x_pos] # has the count of each value in the same order as x_pos


plt.bar(y_pos, counts, align='center', alpha=0.95) # creates the bar plot
plt.xticks(y_pos, x_pos, fontsize = 5)
plt.xlabel('Distance from center peek', fontsize = 5)
plt.ylabel('Counts', fontsize = 5)
plt.title('Distance Counts', fontsize = 5)

ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=90)
ax.tick_params(axis='x', which='major', labelsize=5)
ax.tick_params(axis='y', which='major', labelsize=5)

[l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0] 
# The '2' is the every nth number of labels its shows on the x-axis. So rn is shows every 2nd label. 

plt.gcf().subplots_adjust(bottom = 0.23)
plt.rcParams["figure.dpi"] = 500
print("Total time taken:", time.time() - startTime)
plt.show()

NameError: name 'cube' is not defined